In [1]:
import os
os.environ['LTP_PATH'] = '.'  # local server

import language_tool_python
import pandas as pd

tool = language_tool_python.LanguageTool('en-US', config={'cacheSize': 10000, 'pipelineCaching': True, 'maxCheckThreads': 30, 'warmUp': True})

In [4]:
df = pd.read_csv('wav2vec2-submissions/2337+1781 DSTA raw audiomentations unfreeze val_0.00269 lb 0.01351.csv')

In [5]:
def correct(text):
    match = tool.check(text)
    if match:
        for i, m in enumerate(match):
            if match[i].message == 'Possible spelling mistake found.' and match[i].replacements:
                match[i].replacements[0] = match[i].replacements[0].split()[0]  # prevent it from adding new words
        text = language_tool_python.utils.correct(text, match).upper()
    return text

In [6]:
df['annotation'] = df['annotation'].apply(correct)  # takes 1min 10s

In [5]:
df.to_csv('wav2vec2-submissions/2337+1781 DSTA raw audiomentations unfreeze val_0.00269 lb 0.01351 corrected no word2vec.csv', index=False)

In [ ]:
tool.close()  # doesnt work for me i had to task kill so be careful

In [12]:
text = 'THEY COMPLETE THE CROSSWORD PUZZLE WITHIN FOUR MINUTES WITH NO ERRORSR'
tool.correct(text)

'THEY COMPLETE THE CROSSWORD PUZZLE WITHIN FOUR MINUTES WITH NO ERRORS'

In [47]:
# Fuzzy retrival of digits for finals
import Levenshtein
import numpy as np
digits = ['ZERO', 'ONE', 'TWO', 'THREE', 'FOUR', 'FIVE', 'SIX', 'SEVEN', 'EIGHT', "NINE"]
s = 'SHE WAS EGHT YEARS OLD WHEN SHE LEARNT HOW TO CRY'.split()
digits_dist = []
for digit in digits:
    digits_dist.append([Levenshtein.distance(digit, word) for word in s])
digits_dist = np.array(digits_dist)
min_dist = np.min(np.min(digits_dist, axis=0), axis=0)
min_dist_idx = np.where(digits_dist == min_dist)
if len(min_dist_idx) == 1:  # there is only 1 word that has the lowest distance to any of the digits
    print(f'Replacing {s[min_dist_idx[0][0]]} with {digits[min_dist_idx[0][1]]}')
else:
    print('Multiple words have the same distance to the digits. Choosing 1 based on heuristics')
    common_words = ['TO', 'THE', 'THEY', 'HE', 'SHE']  # if these words are the one that is closest to the digit, then we choose the next one as these are common words and unlikely to be the digit. E.g. TWO -> TO has distance of only 1 but it is most likely wrong
    for i in range(len(min_dist_idx)):
        if s[min_dist_idx[i][0]] in common_words:
            continue
        else:
            print(f'Replacing {s[min_dist_idx[i][0]]} with {digits[min_dist_idx[i][1]]}')
            break

Multiple words have the same distance to the digits. Choosing 1 based on heuristics
Replacing EGHT with EIGHT
Replacing TO with TWO


In [66]:
a = np.array((1, 2, None, None))
# noinspection PyComparisonWithNone
None_idx = np.where(a == None)
print(None_idx[0])
print(a[None_idx])

[2 3]
[None None]
